In [1]:
#we need to import the following libraries.
import pandas as pd
#to display the full text on the notebook without truncation
import re
from snowballstemmer import stemmer
import arabicstopwords.arabicstopwords as stp
#make your loops show a smart progress meter 
from tqdm import tqdm
from glob import glob

In [2]:
#we will use the following function to extract the text from the html files.
def extract_text(file):
    with open(file, 'r', encoding='utf-8') as f:
        text = f.read()
    text = re.sub('<[^<]+?>', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text

In [3]:
html_files = glob('data/*/*.html')

data = pd.DataFrame()

for file in tqdm(html_files):
    text = extract_text(file)
    row = pd.DataFrame({'files': [file], 'raw_texts': [text]})
    data = pd.concat([data, row], ignore_index=True)
    
data.reset_index(drop=True, inplace=True)
data

100%|██████████| 5690/5690 [00:02<00:00, 1933.94it/s]


,files,raw_texts
0,data\Economy\arc_Articlesww0221.html,جمعية الاقتصاديين الدور المأمول حسنا فعلت جمع...
1,data\Economy\arc_Articlesww0313.html,سهم المجموعة العربية للتأمين يرتفع قياسيا ويس...
2,data\Economy\arc_Articlesww03de.html,افتتاح مركز النقد لجيسيك آند ديفريانت في البح...
3,data\Economy\arc_Articlesww04a5.html,أسعار النفط تتراجع في نوفمبر بعد ارتفاعاتها ا...
4,data\Economy\arc_Articlesww0521.html,خلال الفترة بين من الشهر الجاري الاجتماع السن...
...,...,...
5685,data\Sports\arc_Articleszzf992.html,حسين جاسم بطل كمال الأجسام يقول نحن مؤهلون لح...
5686,data\Sports\arc_Articleszzf9e2.html,في ختام بطولة الخليج للاسكواش لاعبو منتخب الك...
5687,data\Sports\arc_Articleszzfb30.html,موقف رياضي ستفجر نتيجة مباراة مصر مع ليبيا في...
5688,data\Sports\arc_Articleszzfc32.html,احتضنته بيت الكرة البحرينية اجتماع تنسيقي إيج...


In [4]:
data['docno'] = [ 'd' + str(i) for i in range(len(data))]
data[['docno']]

,docno
0,d0
1,d1
2,d2
3,d3
4,d4
...,...
5685,d5685
5686,d5686
5687,d5687
5688,d5688


In [5]:
def clean(text):
    text = re.sub(r"http\S+", " ", text) # remove urls
    text = re.sub(r"RT ", " ", text) # remove rt
    text = re.sub(r"@[\w]*", " ", text) # remove handles
    text = re.sub(r"[\.\,\#_\|\:\?\?\/\=]", " ", text) # remove special characters
    text = re.sub(r'\t', ' ', text) # remove tabs
    text = re.sub(r'\n', ' ', text) # remove line jump
    text = re.sub(r"\s+", " ", text) # remove extra white space
    accents = re.compile(r'[\u064b-\u0652\u0640]') # harakaat and tatweel (kashida) to remove

    arabic_punc= re.compile(r'[\u0621-\u063A\u0641-\u064A\d+]+') # Keep only Arabic letters/do not remove number
    text=' '.join(arabic_punc.findall(accents.sub('',text)))
    text = text.strip()
    return text

data['text'] = data['raw_texts'].apply(clean)
data[['docno','raw_texts','text']]

,docno,raw_texts,text
0,d0,جمعية الاقتصاديين الدور المأمول حسنا فعلت جمع...,جمعية الاقتصاديين الدور المأمول حسنا فعلت جمعي...
1,d1,سهم المجموعة العربية للتأمين يرتفع قياسيا ويس...,سهم المجموعة العربية للتأمين يرتفع قياسيا ويسج...
2,d2,افتتاح مركز النقد لجيسيك آند ديفريانت في البح...,افتتاح مركز النقد لجيسيك آند ديفريانت في البحر...
3,d3,أسعار النفط تتراجع في نوفمبر بعد ارتفاعاتها ا...,أسعار النفط تتراجع في نوفمبر بعد ارتفاعاتها ال...
4,d4,خلال الفترة بين من الشهر الجاري الاجتماع السن...,خلال الفترة بين من الشهر الجاري الاجتماع السنو...
...,...,...,...
5685,d5685,حسين جاسم بطل كمال الأجسام يقول نحن مؤهلون لح...,حسين جاسم بطل كمال الأجسام يقول نحن مؤهلون لحص...
5686,d5686,في ختام بطولة الخليج للاسكواش لاعبو منتخب الك...,في ختام بطولة الخليج للاسكواش لاعبو منتخب الكو...
5687,d5687,موقف رياضي ستفجر نتيجة مباراة مصر مع ليبيا في...,موقف رياضي ستفجر نتيجة مباراة مصر مع ليبيا في ...
5688,d5688,احتضنته بيت الكرة البحرينية اجتماع تنسيقي إيج...,احتضنته بيت الكرة البحرينية اجتماع تنسيقي إيجا...


In [6]:
def remove_stopWords(sentence):
    terms=[]
    stopWords= set(stp.stopwords_list())
    for term in sentence.split() : 
        if term not in stopWords :
           terms.append(term)
    return " ".join(terms)

In [7]:
data['text'] = data['text'].apply(remove_stopWords)

In [8]:
def normalize(text):
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    return(text)

In [9]:
data['text'] = data['text'].apply(normalize)

In [10]:
#specify that we want to stem arabic text
ar_stemmer = stemmer("arabic")
#define the stemming function
def stem(sentence):
    return " ".join([ar_stemmer.stemWord(i) for i in sentence.split()])

In [11]:
data['text'] = data['text'].apply(stem)

In [12]:
import pyterrier as pt
if not pt.started():
  pt.init()

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



In [13]:
data[['docno', 'text']]

,docno,text
0,d0,جمع اقتصاد دور مامول حسنا فعل جمع اقتصاد بحرين...
1,d1,سهم مجموعه عربيه تام يرتفع قياسي يسجل مستو جدي...
2,d2,افتتاح مركز نقد لجيس اند ديفريان بحر رعا مءسس ...
3,d3,اسعار نفط تتراجع نوفمبر ارتفاعا قياسيه اكتوبر ...
4,d4,فتره شهر جار اجتماع سنو رابع عشر جنه عربيه رقا...
...,...,...
5685,d5685,حسين جاسم بطل كمال اجسام يقول مءهل لحصد ميدال ...
5686,d5686,ختام طول خليج اسكواش لاعب منتخب كويت يحصد نصيب...
5687,d5687,موقف رياض تفجر نتيج مبار مصر ليبي تصفيا كاس عا...
5688,d5688,احتضن بيت كره بحرينيه اجتماع تنسيق ايجاب لتحضي...


In [14]:
data.head()

,files,raw_texts,docno,text
0,data\Economy\arc_Articlesww0221.html,جمعية الاقتصاديين الدور المأمول حسنا فعلت جمع...,d0,جمع اقتصاد دور مامول حسنا فعل جمع اقتصاد بحرين...
1,data\Economy\arc_Articlesww0313.html,سهم المجموعة العربية للتأمين يرتفع قياسيا ويس...,d1,سهم مجموعه عربيه تام يرتفع قياسي يسجل مستو جدي...
2,data\Economy\arc_Articlesww03de.html,افتتاح مركز النقد لجيسيك آند ديفريانت في البح...,d2,افتتاح مركز نقد لجيس اند ديفريان بحر رعا مءسس ...
3,data\Economy\arc_Articlesww04a5.html,أسعار النفط تتراجع في نوفمبر بعد ارتفاعاتها ا...,d3,اسعار نفط تتراجع نوفمبر ارتفاعا قياسيه اكتوبر ...
4,data\Economy\arc_Articlesww0521.html,خلال الفترة بين من الشهر الجاري الاجتماع السن...,d4,فتره شهر جار اجتماع سنو رابع عشر جنه عربيه رقا...


In [15]:
import os 

# Replace with your desired absolute path for the index
index_path = "C:/Users/AHMED MRABET/Documents/My-Workspace/Information-Retrieval-Arabic/index"

# Create the directory if it doesn't exist
if not os.path.exists(index_path):
    os.makedirs(index_path)

indexer = pt.DFIndexer(index_path, overwrite=True)
indexer.setProperty("tokeniser", "UTFTokeniser")
index_ref = indexer.index(data["text"], data["docno"])
index_ref.toString()

C:\Users\AHMED MRABET\AppData\Local\Temp\ipykernel_10872\3802121502.py:12: DeprecationWarning: Setting of tokeniser property directly is deprecated
  index_ref = indexer.index(data["text"], data["docno"])
c:\Users\AHMED MRABET\AppData\Local\Programs\Python\Python39\lib\site-packages\pyterrier\index.py:628: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, value in meta_column[1].iteritems():


19:03:05.428 [main] ERROR org.terrier.structures.indexing.Indexer - Could not rename index
java.io.IOException: Rename of index structure file 'C:/Users/AHMED MRABET/Documents/My-Workspace/Information-Retrieval-Arabic/index/data_1.direct.bf' (exists) to 'C:/Users/AHMED MRABET/Documents/My-Workspace/Information-Retrieval-Arabic/index/data.direct.bf' (exists) failed - likely that source file is still open. Possible indexing bug?
	at org.terrier.structures.IndexUtil.renameIndex(IndexUtil.java:379)
	at org.terrier.structures.indexing.Indexer.index(Indexer.java:388)
	at org.terrier.structures.indexing.Indexer.index(Indexer.java:355)


'C:/Users/AHMED MRABET/Documents/My-Workspace/Information-Retrieval-Arabic/index/data.properties'

In [16]:
index = pt.IndexFactory.of(index_ref)

In [17]:
tfidf_retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"},num_results=10)

In [18]:
def preprocess(sentence):
    sentence = clean(sentence)
    sentence = remove_stopWords(sentence)
    sentence = normalize(sentence)
    sentence = stem(sentence)

    return sentence

In [19]:
query = "ما هو العمر القانوني للتصويت في الولايات المتحدة؟"
query = preprocess(query)

results=tfidf_retr.search(query)
results

,qid,docid,docno,rank,score,query
0,1,1725,d1725,0,11.472119,عمر قانون تصويت ولا متحده
1,1,1827,d1827,1,10.940176,عمر قانون تصويت ولا متحده
2,1,1755,d1755,2,10.227415,عمر قانون تصويت ولا متحده
3,1,1408,d1408,3,9.783333,عمر قانون تصويت ولا متحده
4,1,1828,d1828,4,9.117229,عمر قانون تصويت ولا متحده
5,1,1664,d1664,5,9.003634,عمر قانون تصويت ولا متحده
6,1,1793,d1793,6,8.800195,عمر قانون تصويت ولا متحده
7,1,1514,d1514,7,8.644252,عمر قانون تصويت ولا متحده
8,1,3752,d3752,8,8.528491,عمر قانون تصويت ولا متحده
9,1,1325,d1325,9,8.380130,عمر قانون تصويت ولا متحده
